In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

general_data = pd.read_csv("data_set/general_data.csv")
employee_survey_data = pd.read_csv("data_set/employee_survey_data.csv")
in_time = pd.read_csv("data_set/in_time.csv")
manager_survey_data = pd.read_csv("data_set/manager_survey_data.csv")
out_time = pd.read_csv("data_set/out_time.csv")
general_data


